# Plurals
As the data were manully inserted in the database, this script supports the entries harmonization through automatic plurals detection. words of the same tipology with a high rank of similarity will be standardized in the format word(plural end of word). They will be then manually corrected in the dataset.  <br>

**Method** <br>
1. For each interested vocabulary, create 2 lists containing all the keys. With a for loop check all the words similarities through the fuzzywuzzy means algorithm
2. identify the difference between similar forms. if the difference is equal to "s", "es", "ies", the words are considered singular/plural. Print the others: if there are plurals, add them manually to the list of tuples afer passage 3.  
3. store the result in a list of tuples (at the beginning inserted as sets to avoid repetitions) where tup[0] = singular, tup[1] = plural, tup[2] = neutral form in "name(s)" type
4. substitute the forms with singular/plural in the dictionary with the new neutral form, adding, if present, the values of the previous keys

## Imports

In [1]:
# !pip install requests
import requests

#!pip install SPARQLWrapper

from SPARQLWrapper import SPARQLWrapper, JSON
import ssl

#!pip install fuzzywuzzy
import difflib
# !pip install python-Levenshtein
# !pip install thefuzz
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
# from thefuzz import fuzz
# from thefuzz import process
import json
import csv
import time #

C:\Users\Sofia Baroncini\AppData\Roaming\Python\Python39\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
from endpoints import *
import functions as f 

In [170]:
# dictionary opening
naturalElementDict = open_json("baseDictionaries/naturalElementDict.json")
characterDict = open_json("Final/characterAlignedFinal.json")

In [9]:
# function that detects plurals, 1) store them in a list of tuples in the format tup[0] = singular, tup [1] = plural, tup[2] = neutral form with (s)
# and 2) update the given dictionary, appending the previous values to the new key in the form "name(s)" and deleting the plural/singular forms
# suggestion: since the dictionary is directly modified, make a copy of it before running
# last point to be solved: shall we modify the link corresponding to the new form? eg. take only the singular, appending it only the first time? in any case the singular form is the first appended value
def detect_plurals(dictionary, plurals_list):
    keys = list(dictionary.keys())
    keys2 = []
    keys3 = []
    doubles = []
    rec_words = list(filter(lambda x: 'rec' in x, keys)) # we select the keys for the recociled terms, to filter them out
    for term in keys: 
        if term not in rec_words:
            keys2.append(term) 
            keys3.append(term) # we create a duplicate list to copare terms to find similarities

    for key in keys3: 
        for key2 in keys2: 
            if key !=key2:
                if fuzz_ratio_mean(key, key2) > 85: # the plural "es" is 87
                    if set((key,key2)) not in doubles:
                        doubles.append(set((key,key2)))
    for s in doubles: 
        couple = tuple(s)
        if len(couple[0])> len(couple[1]): # as it is more frequent, we decide that the longer term is the plural form
            base_term = couple[1]
            plural_term = couple[0]
        elif len(couple[0])< len(couple[1]): 
            base_term = couple[0]
            plural_term = couple[1]
        difference = plural_term.replace(base_term, "")
        pl_addition = "("+str(difference)+")"
        final_form = str(base_term)+pl_addition
        if difference == "s" or difference =="e" or difference =="es": # exeptions: woman(en), man(en), latin plurals (sacophagus. -i)
            plurals_list.append((base_term, plural_term))
        else: 
            print("not included in plurals: ", final_form)
        filtered = filter_values(final)
    return filtered
    



In [10]:
# test
d = {"cicci": "link", "cicci rec":["extLink"], "ciccis": "links", "ciccis rec": ["extLiks"]}
plu = []

final = detect_plurals(d, plu)
print(final)


candidates:  cicci ciccis
Do you like this value?('cicci', 'ciccis')(y/n):y
[('cicci', 'ciccis')]


In [124]:
not_fitted_plurals = []
not_fitted_plurals.append(("sarcophagus", "sarcophagi", "sarcophagus(i)"))
not_fitted_plurals.append(("woman", "women", "woman(women)"))
not_fitted_plurals.append(("man", "men", "man(men)"))
not_fitted_plurals.append(("ox", "oxen", "ox(oxen)"))
not_fitted_plurals.append(("wood", "woods", "wood(s)"))
not_fitted_plurals.append(("kid", "kids", "kid(s)"))
not_fitted_plurals.append(("child", "childdren", "child(dren)"))
not_fitted_plurals.append(("savage", "savages", "savage(s)"))

final = detect_plurals(naturalElementDict2, not_fitted_plurals)

not included in plurals:  acantus plant(acanthus plant)
not included in plurals:  lance(ba)
not included in plurals:  bear(d)
not included in plurals:  ox(b)
not included in plurals:  cart(chart)
not included in plurals:  chariot(eer)
not included in plurals:  child(ren)
not included in plurals:  child(ren)
not included in plurals:  cock(clock)
not included in plurals:  convex lines(concave lines)
not included in plurals:  lowest zone of the tomb(highest zone of the tomb)
not included in plurals:  man face(wo)
not included in plurals:  sarcophagi(sarcophagus)
not included in plurals:  sea(t)
not included in plurals:  shadowed face(shadowened face)
not included in plurals:  statue of a man(statue of a woman)
not included in plurals:  human figure with animal legs(human figures with animal legs)
sarcophagus sarcophagus(i) ['http://icondataset.org/sarcophagi']
women woman(women) ['http://icondataset.org/woman']
men man(men) ['http://icondataset.org/man']
oxen ox(oxen) ['http://icondataset

In [128]:
print(naturalElementDict2["man(men)"])

['http://icondataset.org/man', 'http://icondataset.org/men']


In [72]:
characterDict4 = detect_plurals(characterDict3, plurals_list)

KeyError: 'Angels rec'